# Scenario 2: A cross-functional team with one data scientist working on an ML model
MLflow setup:

- tracking server: yes, local server
- backend store: sqlite database
- artifacts store: local filesystem
The experiments can be explored locally by accessing the local tracking server.

To run this example you need to launch the mlflow server locally by running the following command in your terminal:

`mlflow server --backend-store-uri sqlite:///backend.db`

`mlflow server --backend-store-uri sqlite:///backend.db --default-artifact-root ./artifacts_local`

In [8]:
import mlflow

In [9]:
mlflow.set_tracking_uri('http://127.0.0.1:5000')

In [10]:
print(f"tracking URI : {mlflow.get_tracking_uri()}")

tracking URI : http://127.0.0.1:5000


In [11]:
mlflow.search_experiments()

[<Experiment: artifact_location='/home/nishan/zoomcamp-mlops-learn/02-experiment_tracking/Learning/notebooks/artifacts_local/0', creation_time=1726637842940, experiment_id='0', last_update_time=1726637842940, lifecycle_stage='active', name='Default', tags={}>]

### Creating an experiment and logging a new run

In [12]:
from sklearn.linear_model import LogisticRegression
from sklearn.datasets import load_iris
from sklearn.metrics import accuracy_score

mlflow.set_experiment("my-experiment-1")

with mlflow.start_run():

    X, y = load_iris(return_X_y=True)

    params = {"C": 0.1, "random_state": 42}
    mlflow.log_params(params)

    lr = LogisticRegression(**params).fit(X, y)
    y_pred = lr.predict(X)
    mlflow.log_metric("accuracy", accuracy_score(y, y_pred))

    mlflow.sklearn.log_model(lr, artifact_path="models")
    print(f"default artifacts URI: '{mlflow.get_artifact_uri()}'")

2024/09/18 11:08:14 INFO mlflow.tracking.fluent: Experiment with name 'my-experiment-1' does not exist. Creating a new experiment.
2024/09/18 11:08:18 WARNING mlflow.models.model: Input example should be provided to infer model signature if the model signature is not provided when logging the model.
2024/09/18 11:08:18 INFO mlflow.tracking._tracking_service.client: 🏃 View run legendary-shrimp-684 at: http://127.0.0.1:5000/#/experiments/1/runs/bb105f802b3247f08f24d8c9d377f62b.
2024/09/18 11:08:18 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://127.0.0.1:5000/#/experiments/1.


default artifacts URI: '/home/nishan/zoomcamp-mlops-learn/02-experiment_tracking/Learning/notebooks/artifacts_local/1/bb105f802b3247f08f24d8c9d377f62b/artifacts'


- Tracing will be inside the backend.db
- Artifacts will be inside artifacts_local

In [14]:
mlflow.search_experiments()

[<Experiment: artifact_location='/home/nishan/zoomcamp-mlops-learn/02-experiment_tracking/Learning/notebooks/artifacts_local/1', creation_time=1726637894505, experiment_id='1', last_update_time=1726637894505, lifecycle_stage='active', name='my-experiment-1', tags={}>,
 <Experiment: artifact_location='/home/nishan/zoomcamp-mlops-learn/02-experiment_tracking/Learning/notebooks/artifacts_local/0', creation_time=1726637842940, experiment_id='0', last_update_time=1726637842940, lifecycle_stage='active', name='Default', tags={}>]

### Interacting with the model registry

In [15]:
from mlflow.tracking import MlflowClient


client = MlflowClient("http://127.0.0.1:5000")

In [17]:
client.search_registered_models()

[]

In [38]:
# Registeriung a model
run_id = client.search_runs(experiment_ids='1')[0].info.run_id
mlflow.register_model(
    model_uri=f"runs:/{run_id}/models",
    name='iris-classifier'
)

Successfully registered model 'iris-classifier'.
2024/09/18 11:16:02 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: iris-classifier, version 1
Created version '1' of model 'iris-classifier'.


<ModelVersion: aliases=[], creation_timestamp=1726638362860, current_stage='None', description='', last_updated_timestamp=1726638362860, name='iris-classifier', run_id='bb105f802b3247f08f24d8c9d377f62b', run_link='', source='/home/nishan/zoomcamp-mlops-learn/02-experiment_tracking/Learning/notebooks/artifacts_local/1/bb105f802b3247f08f24d8c9d377f62b/artifacts/models', status='READY', status_message='', tags={}, user_id='', version='1'>